In [1]:
from data_sources.clob import CLOBDataSource
from decimal import Decimal

# Get trading rules and candles
clob = CLOBDataSource()

In [6]:
# Constants
CONNECTOR_NAME = "binance"
INTERVAL = "15m"
DAYS = 20
TOP_X_MARKETS = 20  # Number of top markets to select
# Assuming the `top_markets` DataFrame has been generated as shown in the previous steps
# Constants
TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 200  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

# Example values
TAKE_PROFIT_MULTIPLIER = 0.1  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

# Features configuration
VOLATILITY_WINDOW = 100
TREND_SHORT_WINDOW = 20
TREND_LONG_WINDOW = 100
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 100


# Download data
- Get trading rules
- Get candles for the last x days

In [7]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("FDUSD")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs]

candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

2024-07-26 16:36:12,300 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x17754f250>
2024-07-26 16:36:12,301 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x177fd3b20>, 9711.815747125)]']
connector: <aiohttp.connector.TCPConnector object at 0x17754f1f0>
2024-07-26 16:36:12,327 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x177be9b40>
2024-07-26 16:36:12,328 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x176d0cac0>, 9711.843348)]']
connector: <aiohttp.connector.TCPConnector object at 0x177be9b70>
2024-07-26 16:36:12,339 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1775333d0>
2024-07-26 16:36:12,340 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x177fd178

In [4]:
from features.candles.trend import TrendConfig
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    trend_config=TrendConfig(short_window=TREND_SHORT_WINDOW, long_window=TREND_LONG_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

Error processing RENDER-FDUSD: unsupported operand type(s) for /: 'NoneType' and 'int'


,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
0,MEME-FDUSD,0.00571079,0.0054207,7.31072631,-0.24916016,8296.91471623,0.00021303
1,MANTA-FDUSD,0.00591736,0.00538328,7.95146776,-0.23683039,4414.45390912,0.00012325
2,PEOPLE-FDUSD,0.00816463,0.00942456,10.39737054,0.07519752,24431.49894614,0.00089274
3,AUCTION-FDUSD,0.00522576,0.00352844,5.71243075,-0.0298255,2376.52591454,0.00004766
4,AXL-FDUSD,0.00661558,0.00554105,8.00771079,-0.63735706,2197.03561424,0.00006178
...,...,...,...,...,...,...,...
87,TIA-FDUSD,0.00706046,0.0085228,10.6146323,-0.01406399,15446.2018489,0.00057597
88,AR-FDUSD,0.0062822,0.00524975,8.86095599,-0.50828324,3256.65336467,0.00010129
89,SUI-FDUSD,0.00480009,0.00631109,7.08445994,-0.07220049,13539.71106872,0.00033709
90,INJ-FDUSD,0.00575888,0.00587847,7.53825441,0.36330037,11362.44026826,0.00030087


In [5]:
from research_notebooks.dneitor.utils import filter_top_markets

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=2000, min_atr=0.008, trend_threshold=-0.5)
top_markets

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,normalized_score
56,PEPE-FDUSD,0.00708324,0.01019966,9.83183566,-0.14624427,297950.10276051,0.01030043
22,NOT-FDUSD,0.0070905,0.00996586,9.0372785,-0.24032856,149352.5921606,0.00474724
85,BANANA-FDUSD,0.01140177,0.01466755,12.0166239,0.33237209,95876.82525535,0.00405406
40,WIF-FDUSD,0.00775996,0.01089732,10.36750742,-0.08066665,106248.04667545,0.00387344
61,BONK-FDUSD,0.00736559,0.0098688,10.47911281,-0.14185479,65797.03942603,0.0024235
70,IO-FDUSD,0.00803225,0.01047961,10.61136515,0.13821306,56773.45787558,0.00211791
69,1000SATS-FDUSD,0.01054319,0.01404983,16.64805518,0.09878444,30794.27859171,0.00180126
62,ENA-FDUSD,0.00673268,0.00804745,9.84190527,-0.19589392,51800.45115295,0.00179106
72,WLD-FDUSD,0.00753696,0.01041636,11.4584101,0.0493951,39640.14338593,0.0015963
53,FLOKI-FDUSD,0.00683892,0.0093014,9.11330846,-0.03299587,37237.22636755,0.00119317


In [ ]:
from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml

strategy_config = generate_config(
    id="dneitor-binance_0.4",
    connector_name=CONNECTOR_NAME,
    candles=candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)
dump_dict_to_yaml("configs/", strategy_config)